# Titanic Survivors Example

This expample shows how to train an xgboost classifier and using ONNX converter to pack it in a file compatible with MLKit

## Prerrequisites

In [2]:
import sys

!{sys.executable} -m pip install onnxmltools==1.11.1
!{sys.executable} -m pip install xgboost==1.6.1
!{sys.executable} -m pip install onnxruntime==1.18.1


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


## Modules

Let's import the requried modules

In [3]:
#from onnxconverter_common.onnx_ex import DEFAULT_OPSET_NUMBER
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost  
from onnxmltools.convert.common.data_types import FloatTensorType, StringTensorType,Int32TensorType
from onnxruntime import InferenceSession

from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost  # noqa
import onnxmltools.convert.common.data_types

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

Now, let's get the dataset

In [6]:
df = pd.read_csv("https://gist.githubusercontent.com/fyyying/4aa5b471860321d7b47fd881898162b7/raw/6907bb3a38bfbb6fccf3a8b1edfb90e39714d14f/titanic_dataset.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
df = df.rename(columns={'Pclass': 'TicketClass', 'SibSp': 'NumberOfSiblings', 'Parch': 'NumberOfParents', "Cabin": "CabinNumber", 'Embarked': 'PortOfEmbarkation'})
df = df.drop(['PassengerId', 'Ticket'], axis=1)

# This is a workaround we need for an incompatibility with onnx. It's processors can't handle the np.nan, so the missing data will be filled with null
df['CabinNumber'] = df.CabinNumber.fillna('')
df['PortOfEmbarkation'] = df.PortOfEmbarkation.fillna('')

df

,Survived,TicketClass,Name,Sex,Age,NumberOfSiblings,NumberOfParents,Fare,CabinNumber,PortOfEmbarkation
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,B42,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C148,C


## Feature Engineering

This is one of the most used datasets in the Kaggle space, so we know beforehand some things to do here:

So, we have some missing data. Let's se how can we solve these.

### Categorical Pipeline

For the catecgorical columns,we're going to use a Categorical Imputer for the missing values, a one-hot enconder, and PCA for some dimensionality reduction

In [9]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values='', strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('pca', PCA(n_components=10))
])

### Numerical Pipeline

The numerical pipeline is easier, as we do not need a one hot, we're only to do a KNN based imputation of the numerical value and some scaling

In [10]:
num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

We're set to go, let's build the pipeline.

In [11]:
cat_columns_to_preprocess =  list(df.select_dtypes(['object']).columns)
num_columns_to_preprocess = ['Age','Fare']

In [12]:
# So, yes, this is an actual example of ONNX PreProcessor
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', num_transformer, num_columns_to_preprocess),
        ('categorical', cat_transformer, cat_columns_to_preprocess)
    ])

In [13]:
model = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', XGBClassifier())])

That's it. We've a Pipeline. Let's train it.

## Model Training

Let's train the model

In [14]:
# Let's prepare the dataset
X = df.drop(['Survived'],axis=1)
y = df['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False)),
                                                                  ('pca',
                                                                   PCA(n_comp...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [15]:
# There is no native support for XGBoost, so we need to register one
update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

In [16]:
# Now let's convert it to onnx
# using this instruction set http://www.xavierdupre.fr/app/sklearn-onnx/helpsphinx/auto_examples/plot_pipeline_xgboost.html
model_inputs =[
    ('TicketClass', Int32TensorType([1, 1])),
    ('Name', StringTensorType([1, 1])),
    ('Sex', StringTensorType([1, 1])),
    ('Age', FloatTensorType([1, 1])),
    ('NumberOfSiblings', Int32TensorType([1, 1])),
    ('NumberOfParents', Int32TensorType([1, 1])),
    ('Fare', FloatTensorType([1, 1])),
    ('CabinNumber', StringTensorType([1, 1])),
    ('PortOfEmbarkation', StringTensorType([1, 1]))]

model_onnx = convert_sklearn(
    model, 'pipeline_xgboost',
    model_inputs,
    target_opset={'': 12, 'ai.onnx.ml': 2})
with open("pipeline_xgboost.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())


In [17]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0])

In [18]:
accuracy_score(y_test, y_pred)

0.735593220338983

In [19]:
X_test.iloc[4].values

array([3, 'Nicola-Yarred, Miss. Jamila', 'female', 14.0, 1, 0, 11.2417,
       '', 'C'], dtype=object)